#### Lib Imports

In [22]:
import pandas as pd

#### Load all the CSVs

In [23]:
people = pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\01_people.csv')
abilities = pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\02_abilities.csv')
education = pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\03_education.csv')
experience = pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\04_experience.csv')
person_skills = pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\05_person_skills.csv')
skills = pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\06_skills.csv')

##### Data preview

In [24]:
print("People:", people.columns)
print("Abilities:", abilities.columns)
print("Education:", education.columns)
print("Experience:", experience.columns)
print("Person Skills:", person_skills.columns)
print("Skills:", skills.columns)


People: Index(['person_id', 'name', 'email', 'phone', 'linkedin'], dtype='object')
Abilities: Index(['person_id', 'ability'], dtype='object')
Education: Index(['person_id', 'institution', 'program', 'start_date', 'location'], dtype='object')
Experience: Index(['person_id', 'title', 'firm', 'start_date', 'end_date', 'location'], dtype='object')
Person Skills: Index(['person_id', 'skill'], dtype='object')
Skills: Index(['skill'], dtype='object')


###  Merging Key Resume Fields

##### Merge person_skills with skill names

In [25]:
person_skills['skill'] = person_skills['skill'].fillna('').astype(str)
skills_grouped = person_skills.groupby('person_id')['skill'].apply(lambda x: ', '.join(x)).reset_index()
skills_grouped

,person_id,skill
0,1,"Database administration, Database, Ms sql serv..."
1,2,"sql server management studio, visual studio, s..."
2,3,"DATABASES, ORACLE (4 years), ORACLE 10G, SQL, ..."
3,4,Maintain multiple database environments (Redsh...
4,5,"Scrum, Agile software development, Product bac..."
...,...,...
54853,54929,"Django, Angular JS, JavaScript, JQuery, Node.j..."
54854,54930,"Python, Django, AWS, AngularJS, Bootstrap, Jav..."
54855,54931,"Python 2.7, HTML5, CSS3, AJAX, JSON, JQuery, A..."
54856,54932,"Python 3.1x, PyQuery, PyQt, Django, Angular.js..."


##### Group abilities

In [26]:
abilities_grouped = abilities.groupby('person_id')['ability'].apply(lambda x: ', '.join(x)).reset_index()

##### Get latest education per person

In [27]:
# Fill missing values before sorting
education['program'] = education['program'].fillna('Unknown Program')
education['institution'] = education['institution'].fillna('Unknown Institution')
education['start_date'] = education['start_date'].fillna('1900-01-01')

# Convert start_date to datetime 
education['start_date'] = pd.to_datetime(education['start_date'], errors='coerce')

C:\Users\ABHISHEK S\AppData\Local\Temp\ipykernel_7216\588512914.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  education['start_date'] = pd.to_datetime(education['start_date'], errors='coerce')


In [28]:
education_sorted = education.sort_values(by='start_date')
latest_education = education_sorted.groupby('person_id').last().reset_index()
latest_education['education'] = latest_education['program'] + ' at ' + latest_education['institution']
education_clean = latest_education[['person_id', 'education']]


education_clean


,person_id,education
0,1,Bachelor of Science at Lead City University
1,2,bsc in computer science at lagos state university
2,3,Master of Computer Applications in Science and...
3,4,Bachelor in Computer Science at University of ...
4,5,Unknown Program at Virginia Commomwealth Unive...
...,...,...
48070,54923,Master's in Information technology at Wilmingt...
48071,54924,Bachelor's Degree in Computer Science at JNT U...
48072,54925,Masters in Electrical and Electronics Engineer...
48073,54927,Bachelor of Science in Computers at Andhra Uni...


##### Merge experience (latest job title only)


In [29]:
experience_sorted = experience.sort_values(by='start_date')
latest_experience = experience_sorted.groupby('person_id').last().reset_index()
experience_clean = latest_experience[['person_id', 'title']]
experience_clean.rename(columns={'title': 'last_job_title'}, inplace=True)

C:\Users\ABHISHEK S\AppData\Local\Temp\ipykernel_7216\389782430.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experience_clean.rename(columns={'title': 'last_job_title'}, inplace=True)


##### Combine everything

In [30]:
resume_df = people.merge(skills_grouped, on='person_id', how='left') \
                  .merge(abilities_grouped, on='person_id', how='left') \
                  .merge(education_clean, on='person_id', how='left') \
                  .merge(experience_clean, on='person_id', how='left')

In [31]:
resume_df.head()

,person_id,name,email,phone,linkedin,skill,ability,education,last_job_title
0,1,Database Administrator,NaN,NaN,NaN,"Database administration, Database, Ms sql serv...","Installation and Building Server, Running Back...",Bachelor of Science at Lead City University,Database Administrator
1,2,Database Administrator,NaN,NaN,NaN,"sql server management studio, visual studio, s...","database management systems administration, de...",bsc in computer science at lagos state university,Database Administrator
2,3,Oracle Database Administrator,NaN,NaN,NaN,"DATABASES, ORACLE (4 years), ORACLE 10G, SQL, ...","Over 4+ years of Experience as Architecture, E...",Master of Computer Applications in Science and...,Oracle Database Administrator
3,4,Amazon Redshift Administrator and ETL Develope...,NaN,NaN,NaN,Maintain multiple database environments (Redsh...,"SQL management, PostgresSQL, Oracle, MySQL, mi...",Bachelor in Computer Science at University of ...,Database Administrator
4,5,Scrum Master Scrum Master Scrum Master,NaN,NaN,NaN,"Scrum, Agile software development, Product bac...","Scrum Master, Agile software development, Prod...",Unknown Program at Virginia Commomwealth Unive...,Scrum Master


##### Rename and reorder columns

In [32]:
resume_df = resume_df[['person_id', 'name', 'email', 'phone', 'linkedin',
                       'skill', 'ability', 'education', 'last_job_title']]

In [33]:
resume_df.columns = ['person_id', 'name', 'email', 'phone', 'linkedin',
                     'skills', 'abilities', 'education', 'last_job_title']

In [34]:
resume_df.dropna(subset=['skills'], inplace=True)

In [35]:
resume_df['resume_text'] = (
    resume_df['skills'].fillna('') + ' ' +
    resume_df['abilities'].fillna('') + ' ' +
    resume_df['education'].fillna('') + ' ' +
    resume_df['last_job_title'].fillna('')
)


### Load the Job Listings CSV

In [36]:
main_df= pd.read_csv(r'D:\Coding\Resume_matcher_project\Data\job_postings.csv')
df_skills=pd.read_csv(r"D:\Coding\Resume_matcher_project\Data\job_skills.csv")
df_summary=pd.read_csv(r"D:\Coding\Resume_matcher_project\Data\job_summary.csv")

Index(['job_link', 'last_processed_time', 'last_status', 'got_summary',
       'got_ner', 'is_being_worked', 'job_title', 'company', 'job_location',
       'first_seen', 'search_city', 'search_country', 'search_position',
       'job_level', 'job_type'],
      dtype='object')

In [37]:
job_df= main_df.merge(df_skills, on='job_link', how='left') \
               .merge(df_summary, on='job_link', how='left')

In [54]:
job_df.columns

Index(['job_link', 'last_processed_time', 'last_status', 'got_summary',
       'got_ner', 'is_being_worked', 'job_title', 'company', 'job_location',
       'first_seen', 'search_city', 'search_country', 'search_position',
       'job_level', 'job_type', 'job_skills', 'job_summary', 'job_text'],
      dtype='object')

In [39]:
job_df['job_text'] = (
    job_df['job_title'].fillna('') + ' ' +
    job_df['job_summary'].fillna('') + ' ' +
    job_df['job_skills'].fillna('')
)


### TF-IDF + Cosine Similarity Matching

Objective
- We want to compare a candidate’s resume with job descriptions and find the most relevant jobs — automatically.

But How?
- We treat resume text and job text like documents.
Then we use:

- TF-IDF: to convert that text into numeric vectors

- Cosine Similarity: to measure how similar the resume is to each job

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine all resume and job text into one list for fitting the vectorizer
all_text = pd.concat([resume_df['resume_text'], job_df['job_text']], ignore_index=True)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit on all text and transform
tfidf_matrix = vectorizer.fit_transform(all_text)


 What’s happening?
TF-IDF (Term Frequency - Inverse Document Frequency) gives importance to words that are frequent in a document, but not common across all documents.

Example: “Python” in a resume matters more than words like “the”, “in”, “with”.



In [41]:
n_resumes = resume_df.shape[0]
n_jobs = job_df.shape[0]

resume_vectors = tfidf_matrix[:n_resumes]
job_vectors = tfidf_matrix[n_resumes:]


Why this?
- We created one TF-IDF matrix for both resumes and jobs

- Now we split them into two parts:

- One for resumes

- One for jobs



In [42]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(resume_vectors, job_vectors)


What’s Cosine Similarity?
- Measures the angle between two vectors

- Closer the angle (closer to 1), more similar the resume and job are

- Perfect match = 1.0 (or 100%)

In [43]:
top_matches = []

for idx, similarities in enumerate(similarity_matrix):
    top_indices = similarities.argsort()[::-1][:5]  # get top 5 jobs
    matches = [(job_df.iloc[i]['job_title'], round(similarities[i]*100, 2)) for i in top_indices]
    top_matches.append(matches)

resume_df['top_jobs'] = top_matches

What this does:
- For each resume, find the top 5 most similar jobs

- Save job title and similarity % as a list

- Add this info as a new column top_jobs in resume_df

In [46]:
sample_resume = {
    'person_id': 999,
    'name': 'Kavya Iyer',
    'email': 'kavya.iyer@example.com',
    'phone': '+91-9876543210',
    'linkedin': 'linkedin.com/in/kavya-iyer',
    'skills': 'Python, SQL, Pandas, Data Visualization, Machine Learning',
    'abilities': 'Analytical Thinking, Communication, Problem Solving',
    'education': 'B.Tech in Computer Science at VIT University',
    'last_job_title': 'Data Analyst Intern'
}


In [47]:
import pandas as pd

# Convert dictionary to DataFrame row
sample_df = pd.DataFrame([sample_resume])

# Create resume_text field just like others
sample_df['resume_text'] = (
    sample_df['skills'] + ' ' +
    sample_df['abilities'] + ' ' +
    sample_df['education'] + ' ' +
    sample_df['last_job_title']
)

# Append to your existing resume_df
resume_df = pd.concat([resume_df, sample_df], ignore_index=True)


In [49]:
# Get the last added resume (Kavya's)
new_resume_text = resume_df.iloc[-1]['resume_text']

# Transform using the same vectorizer (do NOT fit again)
new_resume_vector = vectorizer.transform([new_resume_text])


In [50]:
# Get cosine similarity with job vectors
new_similarities = cosine_similarity(new_resume_vector, job_vectors).flatten()


In [51]:
top_indices = new_similarities.argsort()[::-1][:5]

top_matches = [(job_df.iloc[i]['job_title'], round(new_similarities[i] * 100, 2)) for i in top_indices]

# Store result back into the last row
resume_df.at[resume_df.index[-1], 'top_jobs'] = top_matches


In [52]:
print(f"\n👤 {resume_df.iloc[-1]['name']} — Top Job Matches:")
for job, score in top_matches:
    print(f"🔹 {job} ({score}%)")



👤 Kavya Iyer — Top Job Matches:
🔹 Data-Intensive Python/SQL Developer with Machine Learning Experience (29.98%)
🔹 Data Scientist (29.88%)
🔹 Sr Engineer - Data Science (27.07%)
🔹 Data Scientist (26.4%)
🔹 Machine Learning Engineer (26.21%)
